In [26]:
import pandas as pd

**Third country nationals found to be illegally present - annual data (rounded)**

---
Units of measure are absolute numbers of persons.

In [27]:
df = pd.read_csv('../../data/illegally_present_third_country_nationals.csv')

C:\Users\izabe\AppData\Local\Temp\ipykernel_17844\1875699432.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/illegally_present_third_country_nationals.csv')


In [28]:
df.head()

,DATAFLOW,LAST UPDATE,freq,reason,apprehen,citizen,sex,age,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Between border crossing points - at land,Andorra,Females,Total,Person,Austria,2021,0.0,NaN,NaN
1,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Between border crossing points - at land,Andorra,Females,Total,Person,Austria,2022,0.0,NaN,NaN
2,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Between border crossing points - at land,Andorra,Females,Total,Person,Belgium,2021,0.0,NaN,NaN
3,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Between border crossing points - at land,Andorra,Females,Total,Person,Belgium,2022,0.0,NaN,NaN
4,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Between border crossing points - at land,Andorra,Females,Total,Person,Belgium,2023,0.0,NaN,NaN


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12943111 entries, 0 to 12943110
Data columns (total 14 columns):
 #   Column       Dtype  
---  ------       -----  
 0   DATAFLOW     object 
 1   LAST UPDATE  object 
 2   freq         object 
 3   reason       object 
 4   apprehen     object 
 5   citizen      object 
 6   sex          object 
 7   age          object 
 8   unit         object 
 9   geo          object 
 10  TIME_PERIOD  int64  
 11  OBS_VALUE    float64
 12  OBS_FLAG     object 
 13  CONF_STATUS  float64
dtypes: float64(2), int64(1), object(11)
memory usage: 1.4+ GB


In [30]:
df.duplicated().sum()

np.int64(0)

In [31]:
df.isnull().sum()

DATAFLOW              0
LAST UPDATE           0
freq                  0
reason                0
apprehen              0
citizen               0
sex                   0
age                   0
unit                  0
geo                   0
TIME_PERIOD           0
OBS_VALUE         55649
OBS_FLAG       12847425
CONF_STATUS    12943111
dtype: int64

In [ ]:
df = df.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "apprehen", "unit", "OBS_FLAG", "CONF_STATUS"])

In [52]:
df = df[df['OBS_VALUE'] != 0]
df = df[df['geo'] != 'European Union - 27 countries (from 2020)']
df = df[df['geo'] != 'European Union - 28 countries (2013-2020)']

In [ ]:
for col in df:
    unique_vals = df[col].unique()
    print(f"{col}: {unique_vals[:40]}")
    if len(unique_vals) > 40:
        print(f"... and {len(unique_vals) - 40} others")

reason: ['Illegal entry' 'Overstay' 'Other reason' 'Total']
citizen: ['United Arab Emirates' 'Afghanistan' 'Albania' 'Armenia' 'Angola' 'Asia'
 'Azerbaijan' 'Bosnia and Herzegovina' 'Bangladesh' 'Burkina Faso'
 'Burundi' 'Benin' 'Bolivia' 'Belarus' 'Democratic Republic of the Congo'
 'Central African Republic' 'Congo' 'Côte d’Ivoire' 'Cameroon' 'China'
 'Cuba' 'Djibouti' 'Dominican Republic' 'Algeria' 'Ecuador' 'Egypt'
 'Western Sahara' 'Eritrea' 'Ethiopia' 'Georgia' 'Ghana' 'Gambia, The'
 'Guinea' 'Guinea-Bissau' 'Croatia' 'Haiti' 'Indonesia' 'Israel' 'India'
 'Iraq']
... and 141 more
sex: ['Males' 'Total' 'Females' 'Unknown']
age: ['Total' 'Unknown' 'From 14 to 17 years' 'From 18 to 34 years'
 '35 years or over' 'Less than 14 years' 'Less than 18 years']
geo: ['Hungary' 'Bulgaria' 'Cyprus' 'Greece' 'Croatia' 'Latvia' 'Poland'
 'Slovenia' 'Slovakia' 'Romania' 'Montenegro' 'Estonia' 'Finland'
 'Austria' 'Belgium' 'Switzerland' 'Czechia' 'Germany' 'Denmark' 'Spain'
 'Ireland' 'Iceland' 

In [59]:
columns_to_count = ['reason', 'citizen', 'sex', 'age']

for col in columns_to_count:
    print(f"\nCounts for {col}:")
    print(df[col].value_counts())


Counts for reason:
reason
Total            363987
Illegal entry     84759
Other reason      62131
Overstay          57252
Name: count, dtype: int64

Counts for citizen:
citizen
Croatia                           47220
Total                             22146
Asia                              12755
Türkiye                           11525
Syria                             11420
                                  ...  
Samoa                                 4
Federated States of Micronesia        4
Andorra                               2
Marshall Islands                      2
Nauru                                 2
Name: count, Length: 181, dtype: int64

Counts for sex:
sex
Total      223186
Males      200188
Females    128642
Unknown     16113
Name: count, dtype: int64

Counts for age:
age
Total                  171527
From 18 to 34 years    138445
35 years or over       120310
Less than 18 years      48921
From 14 to 17 years     38138
Less than 14 years      34924
Unknown                

In [ ]:
nan_df = df[df.isna().any(axis=1)]
nan_df

grouped = nan_df.groupby(['citizen', 'geo']).size().reset_index(name='count')

table = grouped.pivot(index='citizen', columns='geo', values='count').fillna(0)

print("Count of NaN Observations by Citizen and Geo:")
table
# Iterate through the pivot table and print only non-zero counts
for citizen, row in table.iterrows():
    for geo, count in row.items():
        if count != 0:
            print(f"Citizen: {citizen}, Geo: {geo} -> Count: {int(count)}")


Count of NaN Observations by Citizen and Geo:
Citizen: Africa, Geo: France -> Count: 24
Citizen: Africa, Geo: Sweden -> Count: 48
Citizen: America, Geo: France -> Count: 24
Citizen: America, Geo: Sweden -> Count: 48
Citizen: Asia, Geo: France -> Count: 24
Citizen: Asia, Geo: Netherlands -> Count: 1
Citizen: Asia, Geo: Sweden -> Count: 48
Citizen: Croatia, Geo: Austria -> Count: 852
Citizen: Croatia, Geo: Belgium -> Count: 1344
Citizen: Croatia, Geo: Bulgaria -> Count: 1344
Citizen: Croatia, Geo: Croatia -> Count: 1344
Citizen: Croatia, Geo: Cyprus -> Count: 1344
Citizen: Croatia, Geo: Czechia -> Count: 1344
Citizen: Croatia, Geo: Denmark -> Count: 852
Citizen: Croatia, Geo: Estonia -> Count: 1344
Citizen: Croatia, Geo: Finland -> Count: 1344
Citizen: Croatia, Geo: France -> Count: 540
Citizen: Croatia, Geo: Germany -> Count: 1340
Citizen: Croatia, Geo: Greece -> Count: 1342
Citizen: Croatia, Geo: Hungary -> Count: 1344
Citizen: Croatia, Geo: Iceland -> Count: 672
Citizen: Croatia, Geo:

In [54]:
df.to_csv("../../processed_data/illegally_present_third_country_nationals.csv", index=False)